In [1]:
import pickle
import re
from collections import defaultdict
import operator
import spacy
from spacy.symbols import ORTH
from spacymoji import Emoji
from spacy_cld import LanguageDetector
from textacy import preprocess
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from spacy.tokens import Token

Load unpreprocessed issue comments

In [3]:
# unfortunately we cannot provide the file issue_comments.p here because it is too large.
# In collect_issue_comments_from_db.ipynb it can be seen how to scrape issue comments from the ghtorrent database.
clusterdata = pickle.load(open("issue_commments.p", "rb"))
final_partition= pickle.load(open("data/final_partition", "rb"))

### Defining the pipeline for preprocessing the issue comments
- This mainly deals with removing github markup language
- We also replace code snippets with just the string "CODESAMPLE"
- we replace dates with a generic string "DATE_TIME"
- we replace user mentions with the generic string "USER_MENTION"
- we replace hash references with the generic string "HASHREF"
- we replace full sha with the generic string "FULLSHA"
- we replace individual commit ids with the generic string "C_ID"
- we escape emojis so that they can be treated as a single token
- we replace urls with the generic string 'URL'
- we replace E-mails with the generic string 'EMAIL'
- we replace phone number with the generic string 'PHONE'
- we remove quotes


In [2]:
# %load ../wordembeddings/scripts/preprocess_mongo_31007.py


GITEMOJIS=pickle.load(open("../wordembeddings/scripts/data/gitemojis_list.p", "rb"))
def escape_emojis(text):
    """
    escapes github emoji markup so that it can be treated as a single token.
    """
    rep=text
    for i,emo in enumerate(GITEMOJIS):
        if emo in rep:
            rep= rep.replace(emo,' ' + emo + ' ')  
    return ' '.join(rep.split()) 

HASHREF= re.compile(u"#[0-9]+")
USER_MENTION= re.compile(u"@[0-9a-zA-Z_-]+")
CODEREF= re.compile(u"```[\s\S]+?```")
CODEREF2= re.compile(u"`.*?`")
BOLD= re.compile(u"\*\*(.*?)\*\*")
BOLD2= re.compile(u"\_\_(.*?)\_\_")
ITAL= re.compile(u"\*(.*?)\*")
#ITAL2= re.compile(u"\_(.*?)\_")
HEADER = re.compile("#+? ")
C_ID= re.compile("commit [0-9a-f]{5,40}")
DATE_TIME= re.compile("(?:Mon|Tue|Wed|Thu|Fri|Sat|Sun), "
                      "(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) [0-9]{1,2}, [0-9]{4} "
                      "at [0-9]{1,2}:[0-9]{2} (?:AM|PM)")
FULLSHA=re.compile("[0-9a-f]{40}")
URL_MARKUP=re.compile("\[(.*?)\]\((.*?)\)")


def strip_url_markup(text):
    text= URL_MARKUP.sub(r"\1", text)
    return text

def repl_sha(text):
    """ Replace FULLSHA"""
    return FULLSHA.sub(u"FULLSHA", text)

def strip_hash(text):
    """ strip of header markup"""
    return HEADER.sub(u"", text)

def repl_commit_id(text):
    """ Replace commit ID with C_ID"""
    return C_ID.sub(u"C_ID", text)

def strip_references(text):
    return HASHREF.sub(u"HASHREF", text)

def strip_code(text):
    text=CODEREF.sub(u"CODESAMPLE", text)
    text=CODEREF2.sub(u"CODESAMPLE", text)
    return text
    
def strip_bold(text):
    text= BOLD.sub(r"\1", text)
    text= BOLD2.sub(r"\1", text)
    return text

def strip_italic(text):
    text= ITAL.sub(r"\1", text)
    #text= ITAL2.sub(r"\1", text)
    return text

def repl_date(text):
    text= DATE_TIME.sub("DATE_TIME", text)
    return text
def repl_user_mention(text):
    text= USER_MENTION.sub("USER_MENTION", text)
    return text


SENT_ENDS = [u".", u"!", u"?"]

def tokenize_sentence_split(text, nlp):
    for line in text.split("\n"):
        line= " ".join(line.split())
        tok_acc = []
        doc=nlp(line)
        if len(doc) > 1 and 'en' in doc._.language_scores and doc._.language_scores['en']>0.8:
            entity_text=""
            entity_detected=0
            for tok in doc:
                if tok.ent_iob_=='O':
                    if entity_detected==1:
                        tok_acc.append(entity_text)
                        entity_detected=0      
                    tok_acc.append(tok.text)
                    
                elif tok.ent_iob_=='B':
                    entity_detected=1
                    entity_text=tok.text
                elif tok.ent_iob_=='I':
                    entity_text+='_'+tok.text

                    
                if tok.text in SENT_ENDS:
                    yield " ".join(tok_acc)
                    tok_acc = []
            if tok_acc:
                if entity_detected==1:
                    tok_acc.append(entity_text)              
                yield " ".join(tok_acc)
        else:
             yield doc.text
            


def clean_lines(txt):
    for line in txt.split(u"\n"):
        line= line.strip()
        
        if not line.startswith('>'): #exclude quotes
            if line.startswith('- '):
                line=line[2:]
            yield line


def pre_filter(text):
    text = preprocess.fix_bad_unicode(text, normalization='NFC')
    text= strip_url_markup(text)
    text=strip_hash(text)
    text=strip_references(text)
    text=strip_code(text)
    text=strip_bold(text)
    text= strip_italic(text)
    test= strip_url_markup(text)
    text = repl_commit_id(text)
    text.replace('\r', '') 
    text=repl_sha(text)
    text = preprocess.replace_urls(text, replace_with='URL')
    text = preprocess.replace_emails(text, replace_with='EMAIL')
    text = preprocess.replace_phone_numbers(text, replace_with='PHONE')
    text = repl_user_mention(text)
    text = repl_date(text)
    return text




def extract_text(content, nlp):
    content=pre_filter(content)
    sentences = []
    lines = clean_lines(content)
    for line in lines:
        for sent in tokenize_sentence_split(line, nlp):
            sentences.append(sent)
    return u"\n".join(sentences)


#set up language pipe
nlp = spacy.load('en_core_web_sm', disable=['parser'])
emoji = Emoji(nlp)
nlp.add_pipe(emoji, first=True)
language_detector = LanguageDetector()
nlp.add_pipe(language_detector)

# add special case rule
for emoji in GITEMOJIS:
    nlp.tokenizer.add_special_case(emoji, [{ORTH: emoji}])






Preprocess issue comments

In [ ]:
clustertexts_processed=defaultdict(list)
for k,v in final_partition.items():
    print(k)
    comms = clusterdata[k]
    for obj in comms:
        try:
            clustertexts_processed[v].append(extract_text(obj['comment']['body'],nlp))
        except:
            pass
    

In [3]:
#pickle.dump(clustertexts_processed, open("clustertexts_processed.p", "wb"))  
clustertexts_processed = pickle.load(open("clustertexts_processed.p", "rb"))  

In [4]:
STOP_WORDS = {'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'could',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'former',
 'formerly',
 'forty',
 'four',
 'from',
 'front',
 'full',
 'further',
 'get',
 'give',
 'go',
 'had',
 'has',
 'have',
 'hence',
 'here',
 'hereafter',
 'hereby',
 'herein',
 'hereupon',
 'how',
 'however',
 'hundred',
 'if',
 'in',
 'indeed',
 'into',
 'is',
 'it',
 'its',
 'itself',
 'just',
 'keep',
 'last',
 'latter',
 'latterly',
 'least',
 'less',
 'made',
 'make',
 'many',
 'may',
 'meanwhile',
 'might',
 'more',
 'moreover',
 'most',
 'mostly',
 'move',
 'much',
 'must',
 "n't",
 'name',
 'namely',
 'neither',
 'never',
 'nevertheless',
 'next',
 'nine',
 'no',
 'none',
 'noone',
 'nor',
 'not',
 'nothing',
 'now',
 'nowhere',
 'of',
 'off',
 'often',
 'on',
 'once',
 'one',
 'only',
 'onto',
 'or',
 'other',
 'others',
 'otherwise',
 'out',
 'over',
 'part',
 'per',
 'perhaps',
 'please',
 'put',
 'quite',
 'rather',
 're',
 'really',
 'regarding',
 'same',
 'say',
 'see',
 'seem',
 'seemed',
 'seeming',
 'seems',
 'serious',
 'several',
 'show',
 'side',
 'since',
 'six',
 'sixty',
 'so',
 'some',
 'somehow',
 'someone',
 'something',
 'sometime',
 'sometimes',
 'somewhere',
 'still',
 'such',
 'take',
 'ten',
 'than',
 'that',
 'the',
 'then',
 'thence',
 'there',
 'thereafter',
 'thereby',
 'therefore',
 'therein',
 'thereupon',
 'these',
 'third',
 'this',
 'those',
 'though',
 'three',
 'through',
 'throughout',
 'thru',
 'thus',
 'to',
 'too',
 'toward',
 'towards',
 'twelve',
 'twenty',
 'two',
 'under',
 'unless',
 'until',
 'up',
 'upon',
 'us',
 'used',
 'using',
 'various',
 'very',
 'via',
 'was',
 'were',
 'what',
 'whatever',
 'when',
 'whence',
 'whenever',
 'where',
 'whereafter',
 'whereas',
 'whereby',
 'wherein',
 'whereupon',
 'wherever',
 'whether',
 'which',
 'while',
 'whither',
 'who',
 'whoever',
 'whole',
 'whom',
 'whose',
 '.', '-', '!', '?', '_', '#', '\n', '\r', ',', '<', '>',  ':','[', '(', ';', ']',')', '{', '}', '"', "'",
 '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
  '/tr',
  '/tbody',
  '/table',
  'table',
  'thead', '/th',
  '/thead',
  'tbody',
 'p', '/p', 'm', "'s",  '–', 'm',  'cc',
  'td',
  'tr', "\n\n", "0",
  'th', '%', '=',
  'response&lt;modeltype&gt',
  ';',
 've', 'codesample', 'user_mention',
 'why',
 'will',
 'with',
 'within',
 'without',
 'would',
 'yet',
 "n't", 
 "be",
 "so" }

In [5]:


stop_words_getter = lambda token: token.lower_ in STOP_WORDS or token.lemma_ in STOP_WORDS
Token.set_extension('is_stop', getter=stop_words_getter, force=True)  # set attribute with getter

def mylemma(token):
    lemm = token.lemma_
    if lemm == '-PRON-':
        return token.text.lower()
    else:
        return lemm

def lemmatization(texts):
    texts_out = []
    for i, sent in enumerate(texts):
        if i%1000==0:
            print(i)
        doc = nlp(sent) 
        texts_out.append([mylemma(token) for token in doc if token._.is_stop == False])
    return texts_out

In [ ]:
data_lemmatized={}
for k,v in clustertexts_processed.items():
    data_lemmatized[k]= lemmatization(v)

In [2]:
#pickle.dump(data_lemmatized, open("datalemmatized_simplestopwordlist.p", "wb"))  
#data_lemmatized = pickle.load(open("datalemmatized_simplestopwordlist.p", "rb"))  

Create dictionary with sorted word frequencies for each cluster

In [3]:
def default_tokenizer(string):
    return string.split()

def freq_dist(data):
    """
    """
    ngram_vectorizer = CountVectorizer(tokenizer=default_tokenizer, ngram_range=(1, 1), min_df=0.0005) #tokenizer=default_tokenizer,
    X = ngram_vectorizer.fit_transform(data.split('\n'))
    vocab = list(ngram_vectorizer.get_feature_names())
    counts = X.sum(axis=0).A1
    return Counter(dict(zip(vocab, counts)))


freq_dicts=[]
for i in range(5):
    freq_dict=freq_dist(" ".join([" ".join(items) for items in data_lemmatized[i]]))
    freq_dicts.append(freq_dict)
    
sorted_dicts = [sorted(fd.items(), key=operator.itemgetter(1), reverse=True) for fd in freq_dicts]

# remove some additional stop words that have not been captured by the previous data cleansing.
additional_stopwords="r2 nt de sp etc src lot ref x r1 20 /blockquote></details ﹠ una por con como si un se y la b d h2:0 timeout('timed r … /i></summary><blockquote details><summary><i 26 。 25 joão fullsha status](url this_pull_request 2011 2012 2013 2014 2015 2016 2017 2018 2019 01 02 03 04 05 06 07 08 09 → | + /usr * -- --- ... .. * $ ` lo para el que es en election hashref url 00_ 11 12 13 \ / georgios josé ioannis konstantinos oct nov dec john jose maria dimitrios mohamed mohammad nikolaos fernando antónio antonio luis vasileios carlos abdul luís santos manuel pedro athanasios christos 11168 ".split()
sorted_dicts_stop=[[x for x in sd if x[0] not in additional_stopwords] for sd in sorted_dicts]
#pickle.dump(sorted_dicts_stop,open("data/sorted_wordfrequencies.p", "wb"))
#pickle.dump(freq_dicts, open("data/freq_dicts.p", "wb"))